# Pandas实现Merge
Pandas的merge相当于sql的join ,将不同的表，按照key关联到一起
### Merge的用法
pd.merge(left,right,how="inner",left_on=None,right_on=None,left_index=False,right_index=False,sort=True,suffixes=("x","y),copy=True,indicator=False,validate=None)
+ left,right:要merge的dataframe
+ how: join类型，left,right,outer,inner
+ on: join的key,left和right都要有这个key
+ left_on: left的key
+ left_index: 使用index而不是普通的columns做join
+ suffixes: 两个元素的后缀，列有重名自动加后缀

### 实例
+ 理解merge一对一，一对多，多对多的数量对齐关系
+ 理解left join，right join ,inner join ,outer join的区别
+ 如果出现非key字段重名怎么办

In [1]:
import pandas as pd
df_ratings=pd.read_csv("./datas/movielens-1m/ratings.dat",
              sep="::",
              engine="python",
               names="userId,movieId,rating,timestamp".split(",")
              )

In [2]:
df_ratings.head()  # 包含用户名，电影名，评分，和时间戳

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
df_users = pd.read_csv(
    "./datas/movielens-1m/users.dat", 
    sep="::",
    engine='python', 
    names="userId,gender,age,occupation,zip-code".split(",")
)
df_users.head()

,userId,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
df_movies=pd.read_csv("./datas/movielens-1m/movies.dat",
                     sep="::",
                     engine="python",
                     names="movieId,title,genres".split(","))
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# 以movieId为key可以把title和genres合并进来
df_user=pd.merge(
    df_ratings,df_users,left_on="userId",right_on="userId",how="inner")
df_user.head()

,userId,movieId,rating,timestamp,gender,age,occupation,zip-code
0,1,1193,5,978300760,F,1,10,48067
1,1,661,3,978302109,F,1,10,48067
2,1,914,3,978301968,F,1,10,48067
3,1,3408,4,978300275,F,1,10,48067
4,1,2355,5,978824291,F,1,10,48067


In [16]:
# 每部movieId的电影名字和题材合并进来
df_names=pd.merge(
    df_user,df_movies,left_on="movieId",right_on="movieId",how="inner")
df_names.head()  # 就会按照movieId为key合并起来

,userId,movieId,rating,timestamp,gender,age,occupation,zip-code,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


## 总结
以上的merge本质上是1对1的对应关系

## 理解merge时的对应关系
+ one-to-one: 关联的key都是唯一的
    + 比如（学号，姓名）和（学号，年龄）
    + 结果数为1*1
+ one-to-many: 一对多的关系，左边的key唯一，右边的key不唯一
    + 比如（学号，姓名）merge(学号，\[语文成绩，数学成绩，英语成绩\]
    + 结果数为1\*N
    + 少的会随着多的复制
+ many-ti-many: 多对多关系
    + （学号，\[语文成绩、数学成绩、英语成绩\]） merge (学号，\[篮球、足球、乒乓球\])
    + 结果数为M\*N

### 理解left jon, right join 和inner outer
+ left join 左边key出现的，右边都保留
+ right join 右边key出现的，左边都会保留
+ inner 只有共同存在的 才会保留
+ outer 只要key在左边或者右边出现，都会保留

In [17]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K4', 'K5'],
                      'C': ['C0', 'C1', 'C4', 'C5'],
                      'D': ['D0', 'D1', 'D4', 'D5']})

In [18]:
pd.merge(left, right, how='outer')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,NaN,NaN
3,K3,A3,B3,NaN,NaN
4,K4,NaN,NaN,C4,D4
5,K5,NaN,NaN,C5,D5
